In [1]:
import os
import pandas as pd
from typing import Dict, List
import xml.etree.ElementTree as ET

In [2]:
import survey_parser as SP

In [3]:
online_csv_name = 'Online Table.csv'
in_person_csv_name = 'Inperson Table.csv'
metadata_file_name = 'metadata.xml'
directory = r'C:\Users\clark.roll\python coding\code_personal\Data512\Course Project\Part 2\dataset'

# Full paths for each file
online_path = os.path.join(directory, online_csv_name)
in_person_path = os.path.join(directory, in_person_csv_name)
metadata_path = os.path.join(directory, metadata_file_name)

In [4]:
metadata_questions, metadata_options = SP.parse_survey_metadata(metadata_path)

meta_opts_df = pd.DataFrame(metadata_options)

In [5]:
# Load and structure both CSV files
in_person_df = SP.load_and_structure_survey_csv(in_person_path)
online_df = SP.load_and_structure_survey_csv(online_path)

# Add a column to distinguish in-person and online data
in_person_df['survey_type'] = 'in_person'
online_df['survey_type'] = 'online'

# Combine both datasets into one DataFrame
combined_df = pd.concat([in_person_df, online_df], ignore_index=True)

# Apply the metadata mapping to the combined DataFrame
combined_df_mapped = SP.apply_metadata_mapping(combined_df.copy(),
                                               metadata_questions,
                                               metadata_options)

# Display the mapped DataFrame for inspection
combined_df_mapped

,participant_id,Question 2 Option 1,Question 3 Option 1,Question 3 Option 2,Question 3 Option 3,Question 4 Option 1,Question 4 Option 2,Question 4 Option 3,Question 4 Option 4,Question 4 Option 5,...,Question 28 Option 15,Question 29 Option 1,Question 29 Option 2,Question 29 Option 3,Question 29 Option 4,Question 29 Option 5,Question 29 Option 6,Question 29 Option 7,Question 29 Option 8,survey_type
0,Participant 1,62.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in_person
1,Participant 2,62.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in_person
2,Participant 3,25.0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,in_person
3,Participant 4,53.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in_person
4,Participant 5,44.0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,in_person
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2355,Participant 1742,39.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,online
2356,Participant 1743,48.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,online
2357,Participant 1744,21.0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,online
2358,Participant 1745,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,online


In [6]:
question_dfs = SP.split_all_questions(combined_df_mapped)

In [7]:
q3_df = question_dfs['Question 3']
q3_df

,participant_id,survey_type,Question 3 Option 1,Question 3 Option 2,Question 3 Option 3
0,Participant 1,in_person,1,0,0
1,Participant 2,in_person,1,0,0
2,Participant 3,in_person,0,1,0
3,Participant 4,in_person,1,0,0
4,Participant 5,in_person,0,1,0
...,...,...,...,...,...
2355,Participant 1742,online,1,0,0
2356,Participant 1743,online,1,0,0
2357,Participant 1744,online,0,1,0
2358,Participant 1745,online,0,0,0


In [9]:
q3_df = SP.rename_option_columns(q3_df, meta_opts_df)
q3_df

,participant_id,survey_type,A woman,A man,Other
0,Participant 1,in_person,1,0,0
1,Participant 2,in_person,1,0,0
2,Participant 3,in_person,0,1,0
3,Participant 4,in_person,1,0,0
4,Participant 5,in_person,0,1,0
...,...,...,...,...,...
2355,Participant 1742,online,1,0,0
2356,Participant 1743,online,1,0,0
2357,Participant 1744,online,0,1,0
2358,Participant 1745,online,0,0,0


In [12]:
# Define mapping from "Question X" to actual question descriptions from metadata
question_to_column_mapping = {
    'Question 2': 'age',
    'Question 3': 'gender',
    'Question 4': 'race',
    'Question 5': 'zip_code',
    'Question 6': 'education_level',
    'Question 7': 'income',
    'Question 8': 'general_health_status',
    'Question 9': 'outside_activity_engagement',
    'Question 10': 'outside_activity_frequency',
    'Question 11': 'air_quality_notification_received',
    'Question 12': 'seek_air_quality_info',
    'Question 13': 'info_source_for_smoke_notifications',
    'Question 14': 'days_checked_for_smoke_info',
    'Question 15': 'reduced_outdoor_activities_due_to_smoke',
    'Question 16': 'consecutive_days_reduced_outdoor_activity',
    'Question 17': 'min_aqi_reduce_activity',
    'Question 18': 'min_aqi_eliminate_activity',
    'Question 19': 'motivating_info_to_reduce_outdoor_activity',
    'Question 20': 'motivating_message_type_for_mitigation',
    'Question 21': 'motivating_message_content',
    'Question 22': 'preferred_warning_timing',
    'Question 23': 'future_mitigation_actions',
    'Question 24': 'perception_of_smoke_as_hazard',
    'Question 25': 'compare_smoke_with_other_disasters',
    'Question 26': 'consider_evacuating_due_to_smoke',
    'Question 27': 'smoke_related_health_experience',
    'Question 28': 'symptoms_during_smoke_event',
    'Question 29': 'mitigation_strategies_for_health_issues'
}

# Apply this mapping to rename the columns in combined_df
combined_df.rename(columns=question_to_column_mapping, inplace=True)


In [14]:
combined_df

,participant_id,Question 2_Option 1,Question 3_Option 1,Question 3_Option 2,Question 3_Option 3,Question 4_Option 1,Question 4_Option 2,Question 4_Option 3,Question 4_Option 4,Question 4_Option 5,...,Question 28_Option 15,Question 29_Option 1,Question 29_Option 2,Question 29_Option 3,Question 29_Option 4,Question 29_Option 5,Question 29_Option 6,Question 29_Option 7,Question 29_Option 8,survey_type
0,Participant 1,62.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in_person
1,Participant 2,62.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in_person
2,Participant 3,25.0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,in_person
3,Participant 4,53.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in_person
4,Participant 5,44.0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,in_person
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2355,Participant 1742,39.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,online
2356,Participant 1743,48.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,online
2357,Participant 1744,21.0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,online
2358,Participant 1745,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,online


In [ ]:
# Now, `combined_df` has descriptive column names. You can proceed with separating data by category.
# Using the column_name_mapping to create DataFrames by category as before:

# Create a mapping to convert question labels to the appropriate DataFrames
demographic_questions = ['age', 'gender', 'race', 'zip_code', 'education_level', 'income']
activity_questions = ['general_health_status', 'outside_activity_engagement', 'outside_activity_frequency']
air_quality_questions = [
    'air_quality_notification_received', 'seek_air_quality_info', 'info_source_for_smoke_notifications',
    'days_checked_for_smoke_info', 'reduced_outdoor_activities_due_to_smoke', 
    'consecutive_days_reduced_outdoor_activity', 'min_aqi_reduce_activity', 
    'min_aqi_eliminate_activity', 'motivating_info_to_reduce_outdoor_activity',
    'motivating_message_type_for_mitigation', 'motivating_message_content', 'preferred_warning_timing'
]
hazard_questions = [
    'perception_of_smoke_as_hazard', 'compare_smoke_with_other_disasters', 
    'consider_evacuating_due_to_smoke'
]
health_questions = [
    'smoke_related_health_experience', 'symptoms_during_smoke_event', 
    'mitigation_strategies_for_health_issues'
]

# Define function to create category-specific DataFrames
def create_category_df(columns):
    return combined_df[['survey_type'] + columns]


In [13]:
# Generate the separate DataFrames
demographic_df = create_category_df(demographic_questions)
activity_df = create_category_df(activity_questions)
air_quality_df = create_category_df(air_quality_questions)
hazard_df = create_category_df(hazard_questions)
health_df = create_category_df(health_questions)

KeyError: "['age', 'gender', 'race', 'zip_code', 'education_level', 'income'] not in index"

In [9]:
question_dfs = split_all_questions(combined_df_mapped)

In [10]:
question_2_df = question_dfs['Question 2']

In [11]:
question_2_df

,participant_id,survey_type,Question 2 Option 1
0,Participant 1,in_person,62.0
1,Participant 2,in_person,62.0
2,Participant 3,in_person,25.0
3,Participant 4,in_person,53.0
4,Participant 5,in_person,44.0
...,...,...,...
2355,Participant 1742,online,39.0
2356,Participant 1743,online,48.0
2357,Participant 1744,online,21.0
2358,Participant 1745,online,NaN


In [8]:
print("Columns in combined_df:")
print(combined_df.columns)

Columns in combined_df:
Index(['Inperson Table', 'Question 2', 'Question 3', 'Unnamed: 3',
       'Unnamed: 4', 'Question 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9',
       ...
       'Question 29', 'Unnamed: 170', 'Unnamed: 171', 'Unnamed: 172',
       'Unnamed: 173', 'Unnamed: 174', 'Unnamed: 175', 'Unnamed: 176',
       'survey_type', 'Online Table'],
      dtype='object', length=179)


In [10]:
demographic_df

,survey_type,age,gender,race,zip_code,education_level,income
0,in_person,Option 1,Option 1,Option 1,Option 1,Option 1,Option 1
1,in_person,62,1,1,83676,0,0
2,in_person,62,1,1,83646,0,0
3,in_person,25,0,1,83702,0,0
4,in_person,53,1,1,83669,0,0
...,...,...,...,...,...,...,...
2357,online,39,1,1,83646,0,0
2358,online,48,1,1,83703,0,0
2359,online,21,0,1,83706,0,0
2360,online,NaN,0,0,NaN,0,0


In [7]:
# Parse XML metadata to create question-to-column mapping
tree = ET.parse(metadata_path)
root = tree.getroot()

# Mapping question categories to actual column names based on metadata
question_mapping = {
    "Demographic Data": [],
    "Activity Data": [],
    "Air Quality Notification": [],
    "Natural Hazard Questions": [],
    "Health Questions": []
}

# Populate question mapping with the extracted question names
for category in root.findall('.//Summary_of_survey_questions'):
    for element in category:
        if element.tag == "Demographic_Data":
            question_mapping["Demographic Data"].append(element.text)
        elif element.tag == "Activity_Data":
            question_mapping["Activity Data"].append(element.text)
        elif element.tag == "Air_Quality_Notification":
            question_mapping["Air Quality Notification"].append(element.text)
        elif element.tag == "Natural_Hazard_Questions":
            question_mapping["Natural Hazard Questions"].append(element.text)
        elif element.tag == "Health_Questions":
            question_mapping["Health Questions"].append(element.text)

# Column name mapping based on dataset and metadata structure
column_name_mapping = {
    # Demographic Data
    'Age': 'age',
    'Gender': 'gender',
    'Race': 'race',
    'Zip code': 'zip_code',
    'Education level': 'education_level',
    'Income': 'income',
    
    # Activity Data
    'General health status': 'general_health_status',
    'Engagement in outside activities': 'outside_activity_engagement',
    'Frequency of outside activities': 'outside_activity_frequency',
    
    # Air Quality Notification
    'Receiving/Seeking air quality information and its source': 'air_quality_info_source',
    'Frequency of seeking air quality information': 'seek_air_quality_info_frequency',
    'Reducing outside activities': 'reduce_outside_activity_due_to_smoke',
    'Longest period of consecutive days to reduce outside activities': 'consecutive_days_reduce_outside_activity',
    'Minimum air quality index that convinced to reduce/eliminate outside activities': 'min_aqi_reduce_outside_activity',
    'Effective warning content and delivery method': 'effective_warning_content',
    'Timing of warning': 'warning_timing_preference',
    'Future mitigation planning': 'future_mitigation_planning',
    
    # Natural Hazard Questions
    'Perception of smoke as a hazard': 'smoke_hazard_perception',
    'Comparison with other hazards such as tornadoes and hurricanes': 'compare_smoke_other_hazards',
    'Evacuating home to prevent smoke impacts': 'consider_evacuating_due_to_smoke',
    
    # Health Questions
    'Smoke-related health experience': 'smoke_related_health_experience',
    'Type of observed symptoms': 'observed_symptoms',
    'Mitigation strategies to reduce health issues': 'mitigation_strategies_health_issues'
}

# Update question_mapping with actual column names using the column_name_mapping
for category_name, questions in question_mapping.items():
    question_mapping[category_name] = [column_name_mapping.get(question_text, question_text) for question_text in questions]

# Function to create category-specific DataFrames
def create_category_df(category_name, questions):
    # Filter combined_df to include only the specified questions and survey type
    columns = ['survey_type'] + questions  # Uses the mapped column names
    return combined_df[columns]

# Generate separate DataFrames for each question category
demographic_df = create_category_df("Demographic Data", question_mapping["Demographic Data"])
activity_df = create_category_df("Activity Data", question_mapping["Activity Data"])
air_quality_df = create_category_df("Air Quality Notification", question_mapping["Air Quality Notification"])
hazard_df = create_category_df("Natural Hazard Questions", question_mapping["Natural Hazard Questions"])
health_df = create_category_df("Health Questions", question_mapping["Health Questions"])

# Display the separated DataFrames for inspection
print("Demographic DataFrame:")
print(demographic_df.head())
print("Activity DataFrame:")
print(activity_df.head())
print("Air Quality Notification DataFrame:")
print(air_quality_df.head())
print("Natural Hazard Questions DataFrame:")
print(hazard_df.head())
print("Health Questions DataFrame:")
print(health_df.head())

KeyError: "['age', 'gender', 'race', 'zip_code', 'education_level', 'income'] not in index"

In [5]:
# Building out the column_name_mapping based on the actual structure provided in the metadata

column_name_mapping = {
    # Demographic Data
    'Age': 'age',
    'Gender': 'gender',
    'Race': 'race',
    'Zip code': 'zip_code',
    'Education level': 'education_level',
    'Income': 'income',
    
    # Activity Data
    'General health status': 'general_health_status',
    'Engagement in outside activities': 'outside_activity_engagement',
    'Frequency of outside activities': 'outside_activity_frequency',
    
    # Air Quality Notification
    'Receiving/Seeking air quality information and its source': 'air_quality_info_source',
    'Frequency of seeking air quality information': 'seek_air_quality_info_frequency',
    'Reducing outside activities': 'reduce_outside_activity_due_to_smoke',
    'Longest period of consecutive days to reduce outside activities': 'consecutive_days_reduce_outside_activity',
    'Minimum air quality index that convinced to reduce/eliminate outside activities': 'min_aqi_reduce_outside_activity',
    'Effective warning content and delivery method': 'effective_warning_content',
    'Timing of warning': 'warning_timing_preference',
    'Future mitigation planning': 'future_mitigation_planning',
    
    # Natural Hazard Questions
    'Perception of smoke as a hazard': 'smoke_hazard_perception',
    'Comparison with other hazards such as tornadoes and hurricanes': 'compare_smoke_other_hazards',
    'Evacuating home to prevent smoke impacts': 'consider_evacuating_due_to_smoke',
    
    # Health Questions
    'Smoke-related health experience': 'smoke_related_health_experience',
    'Type of observed symptoms': 'observed_symptoms',
    'Mitigation strategies to reduce health issues': 'mitigation_strategies_health_issues'
}

In [6]:
# Update question_mapping with actual column names using descriptive variable names
for category_name, questions in question_mapping.items():
    question_mapping[category_name] = [column_name_mapping.get(question_text, question_text) for question_text in questions]

# Now proceed with separating DataFrames using updated question_mapping
demographic_df = create_category_df("Demographic Data", question_mapping["Demographic Data"])
activity_df = create_category_df("Activity Data", question_mapping["Activity Data"])
air_quality_df = create_category_df("Air Quality Notification", question_mapping["Air Quality Notification"])
hazard_df = create_category_df("Natural Hazard Questions", question_mapping["Natural Hazard Questions"])
health_df = create_category_df("Health Questions", question_mapping["Health Questions"])

# Display for verification
print("Demographic DataFrame:")
print(demographic_df.head())
print("Activity DataFrame:")
print(activity_df.head())
print("Air Quality Notification DataFrame:")
print(air_quality_df.head())
print("Natural Hazard Questions DataFrame:")
print(hazard_df.head())
print("Health Questions DataFrame:")
print(health_df.head())

NameError: name 'create_category_df' is not defined

In [6]:
# Display extracted question mapping
print("Question Mapping from Metadata:")
for category, questions in question_mapping.items():
    print(f"{category}: {questions}")

# Separate data into individual DataFrames by question category
def create_category_df(category, questions):
    # Filter combined_df to include only the specified questions and survey type
    columns = ['survey_type'] + questions  # Adjust column names as per actual dataset
    return combined_df[columns]

# Generate separate DataFrames for each question category
demographic_df = create_category_df("Demographic Data", question_mapping["Demographic Data"])
activity_df = create_category_df("Activity Data", question_mapping["Activity Data"])
air_quality_df = create_category_df("Air Quality Notification", question_mapping["Air Quality Notification"])
hazard_df = create_category_df("Natural Hazard Questions", question_mapping["Natural Hazard Questions"])
health_df = create_category_df("Health Questions", question_mapping["Health Questions"])

Question Mapping from Metadata:
Demographic Data: ['Age', 'Gender', 'Race', 'Zip code', 'Education level', 'Income']
Activity Data: ['General health status', 'Engagement in outside activities', 'Frequency of outside activities']
Air Quality Notification: ['Receiving/Seeking air quality information and its source', 'Frequency of seeking air quality information', 'Reducing outside activities', 'Longest period of consecutive days to reduce outside activities', 'Minimum air quality index that convinced to reduce/eliminate outside activities', 'Effective warning content and delivery method', 'Timing of warning', 'Future mitigation planning']
Natural Hazard Questions: ['Perception of smoke as a hazard', 'Comparison with other hazards such as tornadoes and hurricanes', 'Evacuating home to prevent smoke impacts']
Health Questions: ['Smoke-related health experience', 'Type of observed symptoms', 'Mitigation strategies to reduce health issues']


KeyError: "['Age', 'Gender', 'Race', 'Zip code', 'Education level', 'Income'] not in index"

In [17]:
# 1. Demographic Data
demographic_columns = ['age', 'gender', 'race', 'zip_code', 'education_level', 'income']  # Replace with actual column names
demographic_df = combined_df[['survey_type'] + demographic_columns]
print("Demographic DataFrame:")
print(demographic_df.head())

# 2. Activity Data
activity_columns = ['general_health_status', 'outside_activity', 'activity_frequency']  # Replace with actual column names
activity_df = combined_df[['survey_type'] + activity_columns]
print("Activity DataFrame:")
print(activity_df.head())

# 3. Air Quality Notification
air_quality_columns = [
    'received_notification', 'seek_info', 'info_source_social_media', 'info_source_tv', 
    'info_source_online_news', 'info_source_newspaper', 'info_source_friends_family'
    # Add more based on metadata
]
air_quality_df = combined_df[['survey_type'] + air_quality_columns]
print("Air Quality Notification DataFrame:")
print(air_quality_df.head())

# 4. Health Questions
health_columns = ['experienced_smoke_illness', 'symptoms', 'mitigation_strategies']  # Replace with actual column names
health_df = combined_df[['survey_type'] + health_columns]
print("Health DataFrame:")
print(health_df.head())

# 5. Natural Hazard Questions
hazard_columns = ['consider_smoke_hazard', 'smoke_vs_other_disasters', 'consider_evacuating']  # Replace with actual column names
hazard_df = combined_df[['survey_type'] + hazard_columns]
print("Natural Hazard Questions DataFrame:")
print(hazard_df.head())

KeyError: "['age', 'gender', 'race', 'zip_code', 'education_level', 'income'] not in index"